# Python多线程
## GIL 全局解释器锁
在其他编程环境中，一般对于单核处理器，同时只能有一个任务执行。多核时可以支持多个线程同时执行。但是在 python 中，无论有多少个核同时只能执行一个线程
- 究其原因，这就是由于 GIL 的存在导致的。GIL 的全程是全局解释器，来源是 python 设计之初的考虑，为了数据安全所做的决定。某个线程想要执行，必须先拿到 GIL，我们可以把 GIL 看做是“通行证”，并且在一个 python 进程之中，GIL 只有一个。拿不到线程的通行证，就不允许进入 CPU 执行

- GIL 只在 cpython 中才有，因为 cpython 调用的是 c 语言的原生线程，所以他不能直接操作 cpu，而只能利用GIL 保证同一时间只能有一个线程拿到数据。而其他解释器如 pypy 和 jpython 中没有 GIL

python 多线程对 CPU 密集型与 IO 密集型的代码执行效率是不同的，因为在执行 IO 密集型代码的过程中多线程切换的主要代价是 IO 切换而执行 CPU 密集型代码多线程切换的主要代价是时间切换，通常 IO 切换比时间切换耗时更长

- CPU 密集型代码（各种循环处理、计算等）在这种情况下，由于计算工作多，每个线程都是执行满 100 tick 后再发出 GIL 的释放与再竞争，所以和单线程跑没啥区别，而且线程切换也需要时间，所以 python 下的多线程对 CPU 密集型代码并不友好
        
- IO 密集型代码（文件处理、网络爬虫等频繁文件读写）多线程能够有效提升效率，单线程下 IO 操作会进入 IO 等待，造成不必要的时间浪费，而开启多线程能在线程 A 等待时，自动切换到线程 B，可以不浪费 CPU 的资源，最终效果类似真正的多线程，在 IO 上节省的时间超过在线程切换消耗的时间，因此能提升程序的执行效率，所以 python 的多线程对 IO 密集型代码比较友好



In [1]:
import os
import time
import threading

## 1、普通创建方式

In [2]:
def run(n):
    print('task',n)
    time.sleep(1)
    print('2s')
    time.sleep(1)
    print('1s')
    time.sleep(1)
    print('0s')
    time.sleep(1)

if __name__ == '__main__':
    # target 是要执行的函数对象名，args 是函数对应的参数，以元组的形式存在
    t1 = threading.Thread(target=run, args=('t1',))
    t2 = threading.Thread(target=run, args=('t2',))
    t1.start()
    t2.start()


task t1
task t2


2s
2s
1s1s

0s0s



## 2、自定义线程
- 继承 threading.Thread 来定义线程类，其本质是重构 Thread 类中的 run 方法


In [ ]:
class MyThread(threading.Thread):
    def __init__(self, n):
        super(MyThread, self).__init__()
        self.n = n

    # 线程启动函数 run 必须写
    def run(self):
        print('task', self.n)
        time.sleep(1)
        print('2s')
        time.sleep(1)
        print('1s')
        time.sleep(1)
        print('0s')
        time.sleep(1)

if __name__ == '__main__':
    t1 = MyThread('t1')
    t2 = MyThread('t2')
    t1.start()
    t2.start()

## 3、守护线程

- 使用 setDaemon(True) 把所有的子线程都变成了主线程的守护线程，因此当主线程结束后，子线程也会随之结束，所以当主线程结束后，整个程序就退出了

- 所谓’线程守护’，就是主线程不管守护线程的执行情况，只要是其他非守护线程结束且主线程执行完毕，主线程都会关闭。也就是说:主线程不等待守护线程的执行完再去关闭

- 主线程在其他非守护线程运行完毕后才算运行完毕（守护线程在此时就被回收）。因为主线程的结束意味着进程的结束，进程整体的资源都将被回收，而进程必须保证非守护线程都运行完毕后才能结束

In [4]:
def run(n):
    print('task', n)
    time.sleep(1)
    print('3s')
    time.sleep(1)
    print('2s')
    time.sleep(1)
    print('1s')

if __name__ == '__main__':
    t = threading.Thread(target=run, args=('t1',))
    # 通过执行结果可以看出，设置守护线程之后，当主线程结束时，子线程也将立即结束，不再执行
    t.setDaemon(True)
    print('start')
    t.start()
    print('end')

start
task t1
end


3s
2s
1s


## 4、让守护线程执行结束之后，主线程再结束
- 我们可以使用 join 方法，让主线程等待守护线程执行完毕再结束

In [5]:
def run(n):
    print('task', n)
    time.sleep(2)
    print('5s')
    time.sleep(2)
    print('3s')
    time.sleep(2)
    print('1s')
    

if __name__ == '__main__':
    t=threading.Thread(target=run, args=('t1',))
    # 把子线程设置为守护线程，必须在 start() 之前设置
    t.setDaemon(True)
    print('start')
    t.start()
    # 设置主线程等待子线程结束
    t.join()
    print('end')

start
task t1
5s
3s
1s
end


## 5、多线程共享全局变量
- 线程是进程的执行单元，进程是系统分配资源的最小执行单位，所以在同一个进程中的多线程是共享资源的

In [6]:
g_num = 100
def work1():
    global g_num
    for i in range(3):
        g_num += 1
    print('in work1 g_num is : %d' % g_num)


def work2():
    global g_num
    print('in work2 g_num is : %d' % g_num)


if __name__ == '__main__':
    t1 = threading.Thread(target=work1)
    t1.start()
    time.sleep(1)
    t2=threading.Thread(target=work2)
    t2.start()

in work1 g_num is : 103
in work2 g_num is : 103


## 6、互斥锁（Lock）
- 由于线程之间是进行随机调度，当多个线程同时修改同一条数据时可能会出现脏数据，所以出现了线程锁，即同一时刻只允许一个线程执行某些操作

- 线程锁用于锁定资源，可以定义多个锁，像下面的代码，当需要独占某一个资源时，任何一个锁都可以锁定这个资源，就好比你用不同的锁都可以把这个相同的门锁住一样

- 由于线程之间是进行随机调度的，如果有多个线程同时操作一个对象，如果没有很好地保护该对象，会造成程序结果的不可预期，也称为“线程不安全”，为了防止上面情况的发生，就出现了互斥锁（Lock）

In [ ]:
def work():
    global n
    lock.acquire()
    temp = n
    time.sleep(0.1)
    n = temp - 1
    lock.release()


if __name__ == '__main__':
    lock = threading.Lock()
    n = 100
    l = []
    for i in range(100):
        p = threading.Thread(target=work)
        l.append(p)
        p.start()
    for p in l:
        p.join()

## 7、递归锁：RLcok类的用法和Lock类一模一样，但它支持嵌套。

- RLock 类代表可重入锁（Reentrant Lock）。对于可重入锁，在同一个线程中可以对它进行多次锁定，也可以多次释放。如果使用 RLock，那么 acquire() 和 release() 方法必须成对出现。如果调用了 n 次 acquire() 加锁，则必须调用 n 次 release() 才能释放锁

- 由此可见，RLock 锁具有可重入性。也就是说，同一个线程可以对已被加锁的 RLock 锁再次加锁，RLock 对象会维持一个计数器来追踪 acquire() 方法的嵌套调用，线程在每次调用 acquire() 加锁后，都必须显式调用 release() 方法来释放锁。所以，一段被锁保护的方法可以调用另一个被相同锁保护的方法


In [ ]:
def func(lock):
    global gl_num
    lock.acquire()
    gl_num += 1
    time.sleep(1)
    print(gl_num)
    lock.release()


if __name__ == '__main__':
    gl_num = 0
    lock = threading.RLock()
    for i in range(10):
        t = threading.Thread(target=func, args=(lock,))
        t.start()


## 8、信号量（BoundedSemaphore类）
- 互斥锁同时只允许一个线程更改数据，而 Semaphore 是同时允许一定数量的线程更改数据，比如厕所有 3 个坑，那最多只允许 3 个人上厕所，后面的人只能等里面有人出来了才能再进去


In [ ]:
def run(n,semaphore):
    # 加锁
    semaphore.acquire()
    time.sleep(3)
    print('run the thread:%s\n' % n)
    # 释放
    semaphore.release()


if __name__== '__main__':
    num = 0
    # 最多允许 5 个线程同时运行
    semaphore = threading.BoundedSemaphore(5)
    for i in range(22):
        t = threading.Thread(target=run, args=('t-%s' % i, semaphore))
        t.start()
    while threading.active_count() !=1:
        pass
    else:
        print('----------all threads done-----------')


## 9、python线程事件
用于主线程控制其他线程的执行，事件是一个简单的线程同步对象，其主要提供以下的几个方法：
- clear 将flag设置为 False
- set 将 flag 设置为 True
- is_set 判断是否设置了 flag
- wait 会一直监听 flag，如果没有检测到 flag 就一直处于阻塞状态
- 事件处理的机制：全局定义了一个 flag，
- 当 flag 的值为 False，那么 event.wait() 就会阻塞，
- 当 flag 值为 True，那么 event.wait() 便不再阻塞


In [ ]:
event = threading.Event()

def lighter():
    count = 0
    # 初始，设置为绿灯
    event.set()
    while True:
        if 5 < count <= 10:
            # 红灯，清除标志位
            event.clear()
            print("\33[41;lmred light is on...\033[0m]")
        elif count > 10:
            # 绿灯，设置标志位
            event.set()
            count = 0
        else:
            print('\33[42;lmgreen light is on...\033[0m')

        time.sleep(1)
        count += 1


def car(name):
    while True:
        # 判断是否设置了标志位
        if event.is_set():
            print('[%s] running.....' % name)
            time.sleep(1)
        else:
            print('[%s] sees red light,waiting...' % name)
            event.wait()
            print('[%s] green light is on,start going...' % name)


startTime = time.time()
light = threading.Thread(target=lighter, )
light.start()

car = threading.Thread(target=car, args=('MINT', ))
car.start()
endTime = time.time()
print('用时：', endTime-startTime)